# Test for databank system

In [ ]:
import os
import sys
# Add path to python modules
sys.path.append(os.path.abspath('..')+'/src/')
from MovieTracks import (DiffusionFitter, OffRateFitter, ParticleFinder)

## Off rate example
**Run thresholding on movie movieNo, read and write threshold from excel file**

In [ ]:
xls_file = '/Users/hubatsl/Desktop/SPT/Us/SPT/sample_data/OffRate_DataBaseTest.xlsx'
movieNo = 0
exec(open("/Users/hubatsl/Desktop/SPT/Us/SPT/Python/src/calibrate_feature_finder.py").read())

**Read time stamps for each movie, write to excel file**

In [ ]:
exec(open("/Users/hubatsl/Desktop/SPT/Us/SPT/Python/src/params_to_excel.py").read())

**Run off rate measurements on all movies specified in .xlsx file**

In [ ]:
p_list = []
for i, path in enumerate(xls_db['Folder']):
    if xls_db['MaxSize'][i]==0:
        p = OffRateFitter(path, xls_db['Threshold'][i], parallel=False)
    else:
        p = OffRateFitter(path, xls_db['Threshold'][i], maxsize=xls_db['MaxSize'][i],
                      parallel=False)
    p.analyze()
    p.fit_offRate([1, 2, 3, 4, 5, 6])
    p.append_output_to_csv('/Users/hubatsl/Desktop/SPT/Us/SPT/sample_data/OffRate_DataBaseTest.csv', p.gData())
    p_list.append(p)
    p.pickle_data()

## Diffusion example

In [ ]:
xls_file = '/Users/hubatsl/Desktop/SPT/Us/SPT/sample_data/Diffusion_DataBaseTest.xlsx'
movieNo = 0
exec(open("/Users/hubatsl/Desktop/SPT/Us/SPT/Python/src/calibrate_feature_finder.py").read())

In [ ]:
exec(open("/Users/hubatsl/Desktop/SPT/Us/SPT/Python/src/params_to_excel.py").read())

In [ ]:
d_list = []
for i, path in enumerate(xls_db['Folder']):
    if xls_db['MaxSize'][i]==0:
        d = DiffusionFitter(path, xls_db['Threshold'][i])
    else:
        d = DiffusionFitter(path, xls_db['Threshold'][i], maxsize=xls_db['MaxSize'][i])
    d.analyze()
    d_list.append(d)

In [ ]:
print(d_list[0].D_restricted)
print(d_list[1].D_restricted)